In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.gridspec import GridSpec
import pyscamp as scamp
import stumpy as stump
import utils.memory as mem

In [ ]:
#Common
ts = np.random.rand(10000)
m = 50

In [ ]:
print("Ts ~ ", ts.shape)
print("Expected mp length: ", int(np.floor(ts.shape[0]-m+1)))

In [ ]:
#SCAMP Prev
# Allows checking if pyscamp was built with CUDA and has GPU support.
has_gpu_support = scamp.gpu_supported()
has_gpu_support

In [ ]:
mp_stumpy = stump.gpu_stump(ts, m)

In [ ]:
mp_scamp, index_scamp = scamp.selfjoin(ts, m)

In [ ]:
def plot_mps(ts, mp_stumpy, mp_scamp):
    fig = plt.figure(figsize=(10, 6))
    gs = GridSpec(3, 1, height_ratios=[1, 4, 4])
    # Serie temporal
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(ts, label="Serie Temporal")
    ax1.set_title("Serie Temporal")
    ax1.legend()
    # MPlot
    ax2 = fig.add_subplot(gs[1], sharex=ax1)
    mp_values = mp_stumpy[:, 0].astype(float) # Extraer solo los valores del perfil de similitud
    ax2.imshow(mp_values.reshape(-1, 1).T, aspect='auto', origin='lower', cmap='hot', extent=(0, len(ts), 0, m))
    ax2.set_title("MPlot - Stumpy")
    
    ax3 = fig.add_subplot(gs[2], sharex=ax1)
    mp_values = mp_scamp.astype(float) # Extraer solo los valores del perfil de similitud
    ax3.imshow(mp_values.reshape(-1, 1).T, aspect='auto', origin='lower', cmap='hot', extent=(0, len(ts), 0, m))
    ax3.set_title("MPlot - Scamp")

    plt.tight_layout()
    plt.show()
plot_mps(ts, mp_stumpy, mp_scamp)

### Ejemplo de STUMPY Basics - Analyzing Motifs and Anomalies with STUMP and SCAMP
https://stumpy.readthedocs.io/en/latest/Tutorial_The_Matrix_Profile.html

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

In [ ]:
steam_df = pd.read_csv("https://zenodo.org/record/4273921/files/STUMPY_Basics_steamgen.csv?download=1")
steam_df.head()

In [ ]:
plt.suptitle('Steamgen Dataset', fontsize='30')
plt.xlabel('Time', fontsize ='20')
plt.ylabel('Steam Flow', fontsize='20')
plt.plot(steam_df['steam flow'].values)
plt.show()

In [ ]:
m = 640
fig, axs = plt.subplots(2)
plt.suptitle('Steamgen Dataset', fontsize='30')
axs[0].set_ylabel("Steam Flow", fontsize='20')
axs[0].plot(steam_df['steam flow'], alpha=0.5, linewidth=1)
axs[0].plot(steam_df['steam flow'].iloc[643:643+m])
axs[0].plot(steam_df['steam flow'].iloc[8724:8724+m])
rect = Rectangle((643, 0), m, 40, facecolor='lightgrey')
axs[0].add_patch(rect)
rect = Rectangle((8724, 0), m, 40, facecolor='lightgrey')
axs[0].add_patch(rect)
axs[1].set_xlabel("Time", fontsize='20')
axs[1].set_ylabel("Steam Flow", fontsize='20')
axs[1].plot(steam_df['steam flow'].values[643:643+m], color='C1')
axs[1].plot(steam_df['steam flow'].values[8724:8724+m], color='C2')
plt.show()

In [ ]:
m = 640
ts = steam_df['steam flow']
mp_scamp, index_scamp = scamp.selfjoin(ts, m)
mp_stumpy = stump.gpu_stump(ts, m)

In [ ]:
plot_mps(ts , mp_stumpy, mp_scamp)

In [ ]:
print("Ts ~ ", ts.shape)
print("Expected mp length: ", int(np.floor(ts.shape[0]-m+1)))
print("mp_stumpy ~", mp_stumpy.shape)
print("mp_scamp ~", mp_scamp.shape)
print("mp_index ~", index_scamp.shape)
print(mp_stumpy[:, 0])
print(mp_scamp)

In [ ]:
mp_stumpy_sorted = np.argsort(mp_stumpy[:,0])
mp_stumpy_sorted

In [ ]:
mp_scamp_sorted = np.argsort(mp_scamp)
mp_scamp_sorted

In [ ]:
motif_idx_stumpy = mp_stumpy_sorted[0]
motif_idx_scamp = mp_scamp_sorted[0]
print(f"The motif (according to stumpy) is located at index {motif_idx_stumpy}")
print(f"The motif (according to scamp) is located at index {motif_idx_scamp}")

### --> Tiene sentido que haya salido diferente por tema de decimales... 
> ¿Serán vecinos?

In [ ]:
nearest_neighbor_idx_stumpy = mp_stumpy[motif_idx_stumpy, 1]
nearest_neighbor_idx_scamp = index_scamp[motif_idx_scamp]
print(f"The nearest neighbor (stumpy) is located at index {nearest_neighbor_idx_stumpy}")
print(f"The nearest neighbor (scamp) is located at index {nearest_neighbor_idx_scamp}")

¡Son los vecinos más cercanos en ambos casos!
De hecho, si imprimimos los valores, la z-distancia euclídea es la misma hasta donde podemos ver...

In [ ]:
print(mp_stumpy[:,0][643])
print(mp_stumpy[:,0][8724])
print(mp_scamp[8724])
print(mp_scamp[643])

In [ ]:
mp_stumpy[:,1][643]

In [ ]:
ts_name = 'Steam Flow'
def plt_motifs(ts, mp, ts_name, algorithm, motif_idx, nearest_neighbor_idx):
    fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})
    plt.suptitle('Motif (Pattern) Discovery | ' + algorithm , fontsize='30')

    axs[0].plot(ts.values)
    axs[0].set_ylabel(ts_name, fontsize='20')
    rect = Rectangle((motif_idx, 0), m, 40, facecolor='lightgrey')
    axs[0].add_patch(rect)
    rect = Rectangle((nearest_neighbor_idx, 0), m, 40, facecolor='lightgrey')
    axs[0].add_patch(rect)
    axs[1].set_xlabel('Time', fontsize ='20')
    axs[1].set_ylabel('Matrix Profile', fontsize='20')
    axs[1].axvline(x=motif_idx, linestyle="dashed", color = "black")
    axs[1].axvline(x=nearest_neighbor_idx, linestyle="dashed", color="red")
    axs[1].plot(mp)
    plt.show()
plt_motifs(ts, mp_stumpy[:,0], ts_name, "Stumpy", motif_idx_stumpy, nearest_neighbor_idx_stumpy)
plt_motifs(ts, mp_scamp, ts_name, "Stumpy", motif_idx_scamp, nearest_neighbor_idx_scamp)

## Discord

In [ ]:
discord_idx_stumpy = mp_stumpy_sorted[-1]
discord_idx_scamp = mp_scamp_sorted[-1]
print(f"Stumpy: The discord is located at index {discord_idx_scamp}")
print(f"Scamp: The discord is located at index {discord_idx_scamp}")

In [ ]:
print("Stumpy | Discord: ", mp_stumpy[discord_idx_stumpy])
print("Scamp |  Discord: [", 
      mp_scamp[discord_idx_scamp], index_scamp[discord_idx_scamp], "]")

The subsequence located at this global maximum is also referred to as a discord, novelty, or “potential anomaly”:

In [ ]:
def plot_discord(ts, mp, ts_name, algorithm, discord_idx):
    fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})
    plt.suptitle('Discord (Anomaly/Novelty) Discovery | '+ algorithm, fontsize='30')

    axs[0].plot(ts.values)
    axs[0].set_ylabel(ts_name, fontsize='20')
    rect = Rectangle((discord_idx, 0), m, 40, facecolor='lightgrey')
    axs[0].add_patch(rect)
    axs[1].set_xlabel('Time', fontsize ='20')
    axs[1].set_ylabel('Matrix Profile', fontsize='20')
    axs[1].axvline(x=discord_idx, linestyle="dashed")
    axs[1].plot(mp)
    plt.show()
plot_discord(ts, mp_stumpy[:,0], ts_name, "Stumpy", discord_idx_stumpy)
plot_discord(ts, mp_scamp, ts_name, "Scamp", discord_idx_scamp)

Intentando ver la matriz Distance Profile al completo

In [ ]:
steam_flow = steam_df['steam flow'].values
#No nans
steam_flow = [ 0 if np.isnan(x) else x for x in steam_flow ]

Cogiendo Distance Matrix al completo

In [ ]:
m = 640

In [ ]:
steam_flow = steam_df['steam flow'].values
#No nans
steam_flow = [ 0 if np.isnan(x) else x for x in steam_flow ]

In [ ]:
n = len(steam_flow) 
subsequence_len = m
print("Expected: ", len(steam_flow) - m + 1)

In [ ]:
MPlot_matrix_stumpy = np.empty((n - m + 1, n - m + 1))
MPlot_matrix_scamp = np.empty((n - m + 1, n - m + 1))

In [ ]:
reference_idx = np.random.randint(low=0, high=len(steam_flow) - m)
reference_subseq = steam_flow[reference_idx:reference_idx + m]

In [ ]:
# Calcular el Distance Profile para cada subsecuencia en la serie temporal
distance_matrix_stumpy = np.array([
    stump.core.mass(reference_subseq, steam_flow[i:i + m]) 
    for i in range(len(steam_flow) - m + 1)
])

distance_matrix_scamp = np.array([
    scamp.abjoin(reference_subseq, steam_flow[i:i + m], m)[0]
    for i in range(len(steam_flow) - m + 1)
])

In [ ]:
print(len(steam_flow))
print(distance_matrix_stumpy.shape)
print(distance_matrix_scamp.shape)
print(MPlot_matrix_stumpy.shape)
print(MPlot_matrix_scamp.shape)

In [ ]:
#Calculamos el MPlot
for i in range(n - m + 1):
    ### Calculamos el Distance Profile utilizando el algoritmo MASS / Scamp
    MPlot_matrix_stumpy[i,:] = stump.core.mass(steam_flow[i:i + m], steam_flow)

In [ ]:
def plot_mp(ts, ts_name, MPlot_matrix, algorithm):
    fig = plt.figure(figsize=(10, 10))
    gs = GridSpec(2, 1, height_ratios=[1, 4])

    # Serie temporal
    ax1 = fig.add_subplot(gs[0])
    ax1.plot(ts, label="Serie Temporal")
    ax1.set_title(ts_name + " | " +  algorithm)
    ax1.legend()

    # MPlot
    ax2 = fig.add_subplot(gs[1], sharex=ax1)
    # Utilizar 'imshow' para visualizar la matriz MPlot
    ax2.imshow(MPlot_matrix, aspect='auto', origin='lower', cmap='hot', extent=(0, len(ts) - m, 0, len(ts) - m))
    ax2.set_title("MPlot")
    ax2.set_xlabel('Subsecuencia Inicial')
    ax2.set_ylabel('Subsecuencia Referencia')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_mp(steam_flow, ts_name, MPlot_matrix_stumpy, "Stumpy")

In [ ]:
mp_scamp, _ = scamp.selfjoin(ts, m)

In [ ]:
mp_scamp.shape

In [ ]:
#https://scamp-docs.readthedocs.io/en/latest/pyscamp/_generate/pyscamp.selfjoin_matrix.htm

In [ ]:
ts.shape

In [ ]:
MPlot_matrix_stumpy.shape

In [ ]:
MPlot_matrix_stumpy

In [ ]:
print(np.isnan(steam_flow).any())

In [ ]:
# MPlot_matrix_stumpy[i,:] = stump.core.mass(steam_flow[i:i + m], steam_flow)
MPlot_matrix_scamp = scamp.selfjoin_matrix(
    steam_flow, 
    m, 
    gpus=[],
    mheight = n - m + 1, 
    mwidth = n - m + 1,
    verbose = True,
    pearson = False
)

In [ ]:
MPlot_matrix_scamp.shape

In [ ]:
MPlot_matrix_scamp

In [ ]:
plot_mp(steam_flow, ts_name, MPlot_matrix_scamp, "Scamp")